In [1]:
import pandas as pd
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Load the dataset
dataset = load_dataset("rjac/kaggle-entity-annotated-corpus-ner-dataset")

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(dataset['train'])
df = df.sample(n=2000, random_state=42)
df.head(3)

,sentence_id,tokens,ner_tags
13389,22048,"[The, report, calls, on, President, Bush, and, Congress, to, urge, Chinese, officials, not, to, use, the, global, war, against, terrorism, as, a, pretext, to, suppress, minorities, ', rights, .]","[0, 0, 0, 0, 1, 2, 0, 3, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3035,1273,"[The, construction, on, the, Baku-T'bilisi-Ceyhan, oil, pipeline, ,, the, Baku-T'bilisi-Erzerum, gas, pipeline, ,, and, the, Kars-Akhalkalaki, Railroad, are, part, of, a, strategy, to, capitalize, on, Georgia, 's, strategic, location, between, Europe, and, Asia, and, develop, its, role, as, a, transit, point, for, gas, ,, oil, and, other, goods, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6013,1541,"[The, pact, was, initially, approved, after, discussions, between, President, Bush, and, Peruvian, President, Alan, Garcia, ,, but, Democrats, in, Congress, forced, U.S., officials, to, reopen, negotiations, and, add, stronger, labor, and, environmental, provisions, .]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 7, 1, 2, 2, 0, 0, 3, 0, 3, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [3]:
def process_data(row):
    sentence = " ".join(row['tokens'])
    ner_tags = row['ner_tags']
    
    output_tokens = row['tokens'].copy()
    for i, tag in enumerate(ner_tags):
        if tag != 0:
            if i == 0 or ner_tags[i-1] == 0:
                output_tokens[i] = "<ENT>"
            else:
                output_tokens[i] = "<CONT>"
    
    output_sentence = " ".join(output_tokens)
    input_sentence = f"Entity Extraction: {sentence}"
    return pd.Series([input_sentence, output_sentence], index=['input', 'output'])

train_data = df.apply(process_data, axis=1)
pd.set_option('display.max_colwidth', None)
train_data.head(1)

,input,output
13389,Entity Extraction: The report calls on President Bush and Congress to urge Chinese officials not to use the global war against terrorism as a pretext to suppress minorities ' rights .,The report calls on <ENT> <CONT> and <ENT> to urge <ENT> officials not to use the global war against terrorism as a pretext to suppress minorities ' rights .


In [4]:
from sklearn.model_selection import train_test_split

# Split the dataset into training, testing, and validation sets
train_df, temp_df = train_test_split(train_data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_df, test_size=0.5, random_state=42)

In [5]:
from transformers import T5Tokenizer
from datasets import Dataset
tokenizer = T5Tokenizer.from_pretrained('t5-base')

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
val_dataset = Dataset.from_pandas(val_data)

def encoder(data):
    inputs = data["input"]
    targets = data["output"]
    model_inputs = tokenizer(inputs, max_length=128, padding="max_length", truncation=True)
    labels = tokenizer(text_target=targets, max_length=128, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the tokenization
train_encoded = train_dataset.map(encoder, batched=True)
test_encoded = test_dataset.map(encoder, batched=True)
val_encoded = val_dataset.map(encoder, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [6]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Initialize the model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="steps",  # Added for periodic evaluation
    save_total_limit=1, 
    save_steps=100,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
)

# Train the model
trainer.train()

  0%|          | 0/1250 [00:00<?, ?it/s]

{'loss': 11.0734, 'grad_norm': 37.30490493774414, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 11.52392864227295, 'eval_runtime': 1.3324, 'eval_samples_per_second': 225.161, 'eval_steps_per_second': 28.52, 'epoch': 0.04}
{'loss': 10.476, 'grad_norm': 46.46704864501953, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.08}


  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.007705272175371647, 'eval_runtime': 3.1577, 'eval_samples_per_second': 95.005, 'eval_steps_per_second': 12.034, 'epoch': 4.96}
{'loss': 0.0188, 'grad_norm': 0.2869487702846527, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.007703553885221481, 'eval_runtime': 3.0915, 'eval_samples_per_second': 97.039, 'eval_steps_per_second': 12.292, 'epoch': 5.0}
{'train_runtime': 520.0988, 'train_samples_per_second': 19.227, 'train_steps_per_second': 2.403, 'train_loss': 0.7479060190320015, 'epoch': 5.0}


TrainOutput(global_step=1250, training_loss=0.7479060190320015, metrics={'train_runtime': 520.0988, 'train_samples_per_second': 19.227, 'train_steps_per_second': 2.403, 'total_flos': 1522394726400000.0, 'train_loss': 0.7479060190320015, 'epoch': 5.0})

In [9]:
# Inference 
def complete_sentence(input_text):
    inputs = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(inputs, max_new_tokens=60)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [10]:
input_text = "Entity Extraction: Ken Pu is going to meet Zikun Fu and Chen Yang on Sunday morning."

print("Input: " +input_text)
print("Output: "+complete_sentence(input_text)+"\n")

Input: Entity Extraction: Ken Pu is going to meet Zikun Fu and Chen Yang on Sunday morning.
Output: ENT> CONT> is going to meet ENT> CONT> and ENT> CONT> on ENT> morning.



In [15]:
input_text = "Entity Extraction: Ken Pu and Zikun is going to have a party by the beach with penguins. Chen Yang is coming with Ma HuaTeng tomorrow for a vacation."

print("Input: " +input_text)
print("Output: "+complete_sentence(input_text)+"\n")

Input: Entity Extraction: Ken Pu and Zikun is going to have a party by the beach with penguins. Chen Yang is coming with Ma HuaTeng tomorrow for a vacation.
Output: ENT> CONT> and ENT> is going to have a party by the beach with penguins. ENT> CONT> is coming with ENT> CONT> tomorrow for a vacation.

